In [112]:
# 1. 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [113]:
# 2. 定义超参数
BACH_SIZE = 16 # 每批处理的数据
DEVICE = "cpu"
ECOPHS = 10 # 训练数据集的轮次


In [114]:
# 3. 构建pipeline， 对图像进行处理
pipeline = transforms.Compose([  
    transforms.ToTensor(), # 将图片转为tensor
    transforms.Normalize((0.1307,), (0.3081,))  # 正则化：降低模型复杂度
])

In [115]:
# 4. 下载、加载数据
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)

test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BACH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BACH_SIZE, shuffle=True)

In [116]:
# 显示MINIST的图片
with open("E:\\code\\ML\\pytorch\\data\\MNIST\\raw\\train-images-idx3-ubyte", "rb") as f:
    file = f.read()
image1 = [int(str(item).encode("ascii")) for item in file[16:16+784]]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 18, 18, 18, 126, 136, 175, 26, 166, 255, 247, 127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 36, 94, 154, 170, 253, 253, 253, 253, 253, 225, 172, 253, 242, 195, 64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 49, 238, 253, 253, 253, 253, 253, 253, 253, 253, 251, 93, 82, 82, 56, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 219, 253, 253, 253, 253, 253, 198, 182, 247, 241, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 80, 156, 107, 253, 253, 205, 11, 0, 43, 154, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 1, 154, 253, 90, 0, 

In [117]:
import cv2
import numpy as np
image1_np = np.array(image1)
print(image1_np.shape)

cv2.imwrite("image1_np.jpg", image1_np)

(784,)


True

In [118]:
# 5. 构建网络
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 5) # 1： 灰度图片的通道， 10：输出通道，5： 卷积核
        self.conv2 = nn.Conv2d(10, 20, 3) # 10: 输入通道， 20：输出通道 3：卷积层
        self.fc1 = nn.Linear(20*10*10, 500) # 20*10*10：输入通道， 500：输出通道
        self.fc2 = nn.Linear(500, 10) # 500：输入通道， 10：输出通道

    def forward(self, x):
        input_size = x.size(0) # batch_size * 1 * 28 * 28
        x = self.conv1(x) # 输入：batch * 1 * 28 * 28, 输出：batch*10*24*24 （28-5+1)
        x = F.relu(x) # 激活, 保持share不变
        x = F.max_pool2d(x, 2, 2) # 池化层，降采样，输入：batch*10*24*24， 输出：batch*10*12*12

        x = self.conv2(x)  # 输入：batch*10*12*12， 输出：batch*20*10*10 （12 -3 + 1）
        x = F.relu(x) 

        x = x.view(input_size, -1) # 拉平

        x = self.fc1(x)  # 输入：batch*2000, 输出：batch*500
        x = F.relu(x) # 激活

        x = self.fc2(x) # 输入: batch*500, 输出： batch*10
        
        output = F.log_softmax(x, dim=1) # 计算分类后，每个数字的概率值
        return output

In [119]:
# 6. 定义优化器
model = Digit().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [120]:
# 7. 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到DEVICE上
        data, target = data.to(device), target.to(device)

        # 梯度初始化为0
        optimizer.zero_grad(0)

        # 训练的结果
        output = model(data)

        # 计算损失
        loss = F.cross_entropy(output, target)

        # 找到概率值最大的下标
        pred = output.max(1, keepdim=True)

        # 反向传播
        loss.backward()

        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch: {} \t Loss: {:.6f})".format(epoch, loss.item()))


In [121]:
# 8. 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()

    # 正确率
    correct = 0.0

    # 测试损失
    test_loss = 0.0
    with torch.no_grad(): # 不会计算你梯度，也不会进行反向传播
        for data, target in test_loader:
            # 部署到device上
            data, target = data.to(device), target.to(device)

            # 测试数据
            output = model(data)

            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()

            # 找出概率值最大的下标
            pred = output.max(1, keepdim=True)[1]
            # pred = torch.max(output, dim=1)
            # pred = output.argmax(dim=1)

            # 累计争取率
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print("Test Average loss: {:.4f}, Accuracy:{:.3f}\n".format(test_loss, 100.0 * correct / len(test_loader.dataset)))



In [111]:
# 调用方法进行训练
for epoch in range(1, ECOPHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x2000 and 500x10)